In [ ]:
# Setup Instructions
# ─────────────────────────────────────────────────────────────
# Before running the notebook:
#
# 1. Replace the placeholders below with your own API key:
#    - OpenAI API key -> See Cell 1 (Environment Setup)
#
# 2. Prepare Google Calendar Authentication:
#    - Download your `credentials.json` file from your Google Cloud Console
#      (after setting up an OAuth 2.0 client ID).
#    - Place `credentials.json` in the same directory as this notebook.
#    - See Cell 2 for Google Calendar authentication setup.
#
# 3. Enable the Google Calendar API:
#    - Visit https://console.cloud.google.com/ to create a project and enable the API.
#    - Guide: https://developers.google.com/calendar/api/quickstart/python
#
# NOTE:
# - The authentication cell checks for a local `token.json`.
# - If it does not exist, the notebook will automatically launch a browser
#   to authenticate your Google account and then generate `token.json`.
##
#
#  3. Install required packages if not already available:
# Required installs are handled directly in the notebook — no need for requirements.txt#
# ❗ WARNING:
#       - Never share your notebook with actual API keys.
#       - Use environment variables or secret cells for secure sharing.



#####

In [ ]:
#CHRONOS AI - Autonomous Scheduling Agent
#2025S 136031-1 GenAI for Humanists
#Link to presentation: https://www.canva.com/design/DAGrcl0KXKc/_ZItQKF1WPu93xHZ2q7xTg/edit?utm_content=DAGrcl0KXKc&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton


#This Notebook ended on 27.06.2025 due to reasons outlined below
#- End of the course is not the end of this project but due to major changes,this one will be outdated. For demonstration purposes I've left the last outputs from the Agent.

#First it is necessary to understand how this Notebook's code logic works:

# 1. User enters a natural language prompt
#    Example: "Schedule 3 coding sessions and 2 workouts next week"

# 2. AgentExecutor starts the planning process using system prompt + toolset

# 3. parse_natural_dates tool:
#    - Parses vague date phrases (e.g., "tomorrow", "next week")
#    - Returns concrete dates (e.g., '01-07-2025')

# 4. Agent generates event blocks with:
#    - Titles, dates, default or inferred times, and durations
#    - The returned event blocks follow the GCal. standard to create an Event via GCal API

# 5. filter_conflicting_events tool:
#    - Checks Google Calendar for scheduling conflicts
#    - Splits into `accepted_blocks` (no conflicts) and `conflicting_blocks`

# 6. (Optional) fallback_planner tool:
#    - Suggests up to 2 alternative time slots for conflicting events
#    - Ensures no overlaps and avoids already blocked times

# 7. create_calendar_events tool:
#    - Creates Google Calendar events (main + 15-min buffer)
#    - Returns success/failure messages per event

# 8. Agent summarizes the result:
#    - Breaks down what was planned
#    - Asks for user approval if needed
#    - Executes creation when confirmed

# ────────────────────────────────────────────────────────────────
# End-to-End Flow:
# User Prompt → Date Parsing → Event Generation →
# Conflict Filtering → Fallback Planning (if needed) →
# Event Creation → Agent Response


#Current limitations in this Notebook:
#- Agent Chain Logic: Need a new logic flow for the tools that the Agent calls, his notebook is highly inefficient because the Agent creates Event Blocks for Google Calendar first and THEN cross-checks with the calendar. See steps 4, 5 and 6 above. Need reorder.
#- The Memory is based on tinydb and therefore JSON logic. This is not scalable for further usecases.
#- This project is hitting limitations and is quite costly due to chain logic, System prompt is txt heavy, looping Agent...
#- In this project the Agent fails with the Fallback_planner tool due to UUID (unique_id) issues, this will not be further resolved here because it involves a DB and UUID problem that is quite annoying to resolve. Since I will change Dev. Environment + Database, I have kept it as it is (Agent will most likely return Error when calling fallback_planner_tool after User confirmed to re-schedule overlapping events.)


#####

In [ ]:
#### 📦 Import Cell — All Required Libraries

from openai import OpenAI
from tinydb import TinyDB, Query
fallback_db = TinyDB("fallback_sessions.json")
from dateutil import parser
import pytz
import ast
import re
import uuid
tz = pytz.timezone("Europe/Berlin")
from dateutil.parser import parse as parse_datetime

import os
os.environ["OPENAI_API_KEY"] = "INSERT-API-KEY-HERE"

client = OpenAI()

#New Agent Imports:

from typing import List, Dict, Optional
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool
from pydantic import BaseModel
from typing import List
import dateparser
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
from datetime import date, timedelta, datetime
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


#Optional imports depend if needed such as print

from pprint import pprint

# Initialize chat model
llm = ChatOpenAI(model="gpt-4-0613", temperature=0.0)



#####

/Users/mario/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/wr/6v2y8lz94kx8vgnp3cclfp800000gn/T/ipykernel_1365/1118137446.py:58: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [ ]:
# 🔐 Google Calendar Authentication (without re-direct; see local token.json). If token.json is not there, re-direct should happen automatically.

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import os.path

# Define the OAuth 2.0 scopes
scopes = ['https://www.googleapis.com/auth/calendar']

# Authenticate and create the service
creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', scopes)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', scopes)
        creds = flow.run_local_server(port=0)
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

# Initialize Google Calendar API service
service = build("calendar", "v3", credentials=creds)


#####

In [ ]:
#Date Parser Tool

class DateParseInput(BaseModel):
    phrases: List[str]
today = date.today()

def format_date(d: date) -> str:
    return d.strftime("%d-%m-%Y")

def today_str() -> str:
    return format_date(today)

def get_weekday_on_or_after(target_weekday: int, after_days: int = 0) -> date:
    """
    Get the next target_weekday on or after today + after_days.
    """
    base = today + timedelta(days=after_days)
    days_until = (target_weekday - base.weekday() + 7) % 7
    return base + timedelta(days=days_until)

def get_this_week_dates():
    today_dt = date.today()
    # Find Monday of this week
    monday = today_dt - timedelta(days=today_dt.weekday())
    # List of dates from today through Sunday (skip days before today)
    week_dates = [
        format_date(monday + timedelta(days=i))
        for i in range(7)
        if (monday + timedelta(days=i)) >= today_dt
    ]
    return week_dates

PHRASE_MAP = {
    "today": lambda: [format_date(today)],
    "tomorrow": lambda: [format_date(today + timedelta(days=1))],
    "this weekend": lambda: [
        format_date(get_weekday_on_or_after(5)),  # Saturday
        format_date(get_weekday_on_or_after(6))   # Sunday
    ],
    "next weekend": lambda: [
        format_date(get_weekday_on_or_after(5, after_days=7)),
        format_date(get_weekday_on_or_after(6, after_days=7))
    ],
    "next week": lambda: [
        format_date(get_weekday_on_or_after(i, after_days=7)) for i in range(0, 5)
    ],
    "this week": get_this_week_dates,
    "next monday": lambda: [format_date(get_weekday_on_or_after(0))],
    "next tuesday": lambda: [format_date(get_weekday_on_or_after(1))],
    "next wednesday": lambda: [format_date(get_weekday_on_or_after(2))],
    "next thursday": lambda: [format_date(get_weekday_on_or_after(3))],
    "next friday": lambda: [format_date(get_weekday_on_or_after(4))],
}

@tool(args_schema=DateParseInput)
def parse_natural_dates(phrases: List[str]) -> Dict[str, Dict[str, List[str]]]:
    """
    Parse natural language date expressions like 'this weekend' or 'next Thursday'
    and return a dict mapping each phrase to a list of one or more dates (in DD-MM-YYYY format).
    The result is returned inside a 'parsed_dates' key so the agent can reason properly.
    """
    parsed_dates = {}

    for phrase in phrases:
        key = phrase.lower().strip()

        if key in PHRASE_MAP:
            parsed_dates[phrase] = PHRASE_MAP[key]()
        else:
            parsed = dateparser.parse(
                key,
                settings={"PREFER_DATES_FROM": "future", "RELATIVE_BASE": datetime.now()}
            )
            if parsed:
                parsed_dates[phrase] = [parsed.strftime('%d-%m-%Y')]
            else:
                parsed_dates[phrase] = ["unrecognized"]

    return {"parsed_dates": parsed_dates}




#####

In [ ]:
#Filter Conflicting Events Tool - Filters out new events that clash with existing events and returns them.

from collections import defaultdict

class BlockInput(BaseModel):
    events: List[Dict]

@tool(args_schema=BlockInput)
def filter_conflicting_events(events: List[Dict]) -> dict:
    """
    Filters out events that conflict with existing calendar events.
    An event conflicts if it overlaps with an existing event or its 15-minute post-event buffer.
    """
    blocks = events

    #print("🛠️ Received input in filter_conflicting_events:", blocks)

    if not blocks:
        print("⚠️ No blocks provided to check for conflicts.")
        return {
            "has_conflicts": False,
            "accepted_blocks": [],
            "conflicting_blocks": [],
            "all_events_by_date": {}
        }

    # Get all calendar IDs (primary + subscribed)
    calendar_list = service.calendarList().list().execute()
    calendar_ids = [entry["id"] for entry in calendar_list.get("items", [])]
    time_min = datetime.now(tz).astimezone(pytz.utc).isoformat()
    time_max = (datetime.now(tz) + timedelta(days=7)).astimezone(pytz.utc).isoformat()

    all_events = []
    for calendar_id in calendar_ids:
        try:
            events_resp = service.events().list(
                calendarId=calendar_id,
                timeMin=time_min,
                timeMax=time_max,
                singleEvents=True,
                orderBy="startTime"
            ).execute()
            all_events.extend(events_resp.get("items", []))
        except Exception as e:
            print(f"⚠️ Could not access calendar {calendar_id}: {e}")

    # For conflict checking
    existing_events = [
        {
            "start": parse_datetime(e["start"]["dateTime"]).replace(microsecond=0),
            "end": parse_datetime(e["end"]["dateTime"]).replace(microsecond=0)
        }
        for e in all_events
        if "dateTime" in e["start"] and "dateTime" in e["end"]
    ]

    # For fallback planner: group by date as ISO strings
    all_events_by_date = defaultdict(list)
    for e in all_events:
        if "dateTime" in e["start"] and "dateTime" in e["end"]:
            start_dt = parse_datetime(e["start"]["dateTime"])
            end_dt = parse_datetime(e["end"]["dateTime"])
            date_str = start_dt.strftime("%d-%m-%Y")
            all_events_by_date[date_str].append({
                "start": start_dt.isoformat(),
                "end": end_dt.isoformat()
            })
    all_events_by_date = dict(all_events_by_date)  # make serializable



    accepted_blocks = []
    conflicting_blocks = []

    for block in blocks:
        start = tz.localize(datetime.strptime(f"{block['date']} {block['time']}", "%d-%m-%Y %H:%M")).replace(microsecond=0)
        end = start + timedelta(minutes=block.get("duration_minutes", 60))
        buffered_end = end + timedelta(minutes=15)

        overlapping = [
            event for event in existing_events
            if start < event["end"] and buffered_end > event["start"]
        ]

        if overlapping:
            print(f"⚠️ Conflict: {block['title']} at {block['date']} {block['time']}")
            date_str = block['date']

            block_with_context = block.copy()
            block_with_context["context_id"] = str(uuid.uuid4())

            conflicting_blocks.append({
                "proposed": block_with_context,
                "conflicts_with": [
                    {
                        "start": e["start"].isoformat(),
                        "end": e["end"].isoformat()
                    } for e in overlapping
                ],
                "all_busy_events_for_date": all_events_by_date.get(date_str, [])
})
        else:
            accepted_blocks.append(block)

    print(f"✅ {len(accepted_blocks)} events passed conflict check")
    print(f"[DEBUG] Final conflicting_blocks: {conflicting_blocks}")
    return {
        "has_conflicts": len(conflicting_blocks) > 0,
        "accepted_blocks": accepted_blocks,
        "conflicting_blocks": conflicting_blocks,
    }


#####

In [ ]:
#Fallback Planner Tool - called IF new events overlap with existing events


class FallbackPlannerInput(BaseModel):
    proposed: Optional[Dict] = None  # Now optional, to allow fallback logic
    existing: List[Dict]     # Existing events, with 'start' and 'end' as ISO strings
    available_start: str = "08:00"   # When you want your day to start
    available_end: str = "22:00"     # When you want your day to end
    min_duration: int = 60           # Minimum event duration (in minutes)
    blocked: Optional[List[Dict]] = []  # Previously suggested or taken fallback slots
    accepted_blocks: Optional[List[Dict]] = [] # Include all events into fallback tool
    context_id: Optional[str] = None  # Unique ID like "Coding Session 1_21-06-2025"

@tool(args_schema=FallbackPlannerInput)
def fallback_planner(
    existing: List[Dict],
    proposed: Optional[Dict] = None,
    accepted_blocks: Optional[List[Dict]] = None,
    available_start: str = "08:00",
    available_end: str = "22:00",
    min_duration: int = 60,
    blocked: Optional[List[Dict]] = None,
    context_id: Optional[str] = None
) -> List[Dict]:
    """
    Suggests up to 2 alternative free time slots for a conflicting event on the same day, if possible.
    Avoids returning slots listed in `blocked`.
    """
    blocked = blocked or []
    accepted_blocks = accepted_blocks or []

    # 🔐 STEP 1: Fallback to saved session data if proposed is missing
    if not proposed and context_id:
        session = fallback_db.get(Query().context_id == context_id)
        if session and "proposed" in session:
            proposed = session["proposed"]
            print(f"[FallbackPlanner] ⏪ Restored proposed from memory for context: {context_id}")
        else:
            print("[FallbackPlanner] ❌ No proposed event found for given context_id")
            return []

    if not proposed:
        print("[FallbackPlanner] ❌ No proposed event provided or found. Aborting.")
        return []

    if not context_id:
        context_id = str(uuid.uuid4())  # Truly unique fallback session ID

    # 🔐 STEP 2: Save current session state for future reuse
    fallback_db.upsert({
        "context_id": context_id,
        "title": proposed["title"],
        "date": proposed["date"],
        "proposed": proposed,
        "existing": existing,
        "accepted_blocks": accepted_blocks,
        "blocked": blocked,
        "created_at": datetime.now().isoformat()
}, Query().context_id == context_id)

    date = proposed["date"]
    duration = proposed.get("duration_minutes", min_duration)

    print("[fallback_planner] [debug] Proposed:", proposed)
    print("[fallback_planner] [debug] Existing:", existing)
    print("[fallback_planner] [debug] Blocked:", blocked)

    try:
        available_start_dt = tz.localize(datetime.strptime(f"{date} {available_start}", "%d-%m-%Y %H:%M"))
        available_end_dt = tz.localize(datetime.strptime(f"{date} {available_end}", "%d-%m-%Y %H:%M"))
    except Exception as e:
        print(f"[FallbackPlanner] ❌ Date parsing error: {e}")
        return []

    # Build list of blocked datetime ranges for comparison
    blocked_ranges = []
    for b in blocked:
        try:
            start = tz.localize(datetime.strptime(f"{b['date']} {b['start_time']}", "%d-%m-%Y %H:%M"))
            end = start + timedelta(minutes=b.get("duration_minutes", duration))
            blocked_ranges.append((start, end))
        except:
            continue

    # Parse and sort busy intervals
    busy = []
    for e in existing:
        start = parse_datetime(e["start"])
        end = parse_datetime(e["end"])
        if start.date() == available_start_dt.date():
            busy.append((start, end))
    busy.sort()

    # Find free slots between busy intervals
    free_slots = []
    last_end = available_start_dt
    for start, end in busy:
        if (start - last_end).total_seconds() >= duration * 60:
            free_slots.append((last_end, start))
        last_end = max(last_end, end)
    if (available_end_dt - last_end).total_seconds() >= duration * 60:
        free_slots.append((last_end, available_end_dt))

    # Build suggestions (avoid blocked)
    suggestions = []
    for slot_start, slot_end in free_slots:
        alt_start = slot_start
        alt_end = alt_start + timedelta(minutes=duration)

        # Skip if this slot overlaps with any blocked slot
        overlaps = any(
            alt_start < b_end and alt_end > b_start
            for b_start, b_end in blocked_ranges
        )
        if overlaps:
            continue

        suggestions.append({
            "date": date,
            "start_time": alt_start.strftime("%H:%M"),
            "end_time": alt_end.strftime("%H:%M"),
            "duration_minutes": duration
        })

        if len(suggestions) >= 2:
            break

    return suggestions


#####

In [ ]:
#Create Calendar Events that are cross-checked with existing_events + created in GCal IF User confirms to do so.


class EventInput(BaseModel):
    events: List[Dict]

@tool(args_schema=EventInput)
def create_calendar_events(events: List[Dict]) -> List[str]:
    """
    Creates confirmed events directly in the user's Google Calendar.
    Also adds a 15-minute buffer event after each main event.
    Returns a list of creation summaries.
    """
    timezone = 'Europe/Berlin'
    buffer_minutes = 15
    created_messages = []

    for event in events:
        title = event["title"]
        date = event["date"]
        time = event["time"]
        duration = event.get("duration_minutes", 60)
        color_id = str(event.get("color_id", 11))

        try:
            start_dt = tz.localize(datetime.strptime(f"{date} {time}", "%d-%m-%Y %H:%M"))
            end_dt = start_dt + timedelta(minutes=duration)
            group_id = f"{title}-{date}-{time}"
            gcal_event = {
                "summary": title,
                "start": {"dateTime": start_dt.isoformat(), "timeZone": timezone},
                "end": {"dateTime": end_dt.isoformat(), "timeZone": timezone},
                "colorId": color_id,
                "description": f"group_id:{group_id};type:main"
            }

            service.events().insert(calendarId='primary', body=gcal_event).execute()
            created_messages.append(f"✅ Created: {title} — {date} at {time} for {duration} min")

            # Add post-event buffer
            buffer_event = {
                "summary": "Buffer",
                "start": {"dateTime": end_dt.isoformat(), "timeZone": timezone},
                "end": {"dateTime": (end_dt + timedelta(minutes=buffer_minutes)).isoformat(), "timeZone": timezone},
                "colorId": color_id,
                "description": f"group_id:{group_id};type:buffer"
            }

            service.events().insert(calendarId='primary', body=buffer_event).execute()

        except Exception as e:
            created_messages.append(f"❌ Failed to create {title} on {date} at {time}: {e}")

    return created_messages


#####

In [ ]:
#System Prompt


prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a smart calendar assistant. Your job is to help users plan events in their calendar from natural instructions. You MUST follow the workflow exactly as described. Never guess, assume, or skip tool calls — even if the task seems easy. Every decision MUST be based on validated tool output. If no tool was called, do not propose events or summaries.\n\n"
     "Workflow:\n"
     "1. Understand all time references in the user's request. If no specific time is provided, create one. For vague or relative dates (e.g., 'tomorrow', 'this weekend'), call the `parse_natural_dates` tool and use its output.\n"
     "2. With the resolved dates, create a list of structured events. Each event block must have:\n"
     "   - title\n"
     "   - date (DD-MM-YYYY)\n"
     "   - time (HH:MM)\n"
     "   - duration_minutes (integer)\n"
     "3. Call the `filter_conflicting_events` tool using your events. This tool returns:\n"
     "   - has_conflicts (boolean)\n"
     "   - accepted_blocks (list of events that do not conflict)\n"
     "   - conflicting_blocks (list of dicts, each with 'proposed' and 'conflicts_with')\n"
     "   - all_events_by_date (dictionary mapping dates to all events for that day)\n"
     "4. Analyze the result from `filter_conflicting_events`:\n"
     "   - If has_conflicts is false:\n"
     "       - Present accepted_blocks to the user, and ask for confirmation to schedule.\n"
     "       - Example: 'Great! These events are conflict-free. Shall I add them to your calendar?'\n"
     "   - If has_conflicts is true:\n"
     "       - For each item in conflicting_blocks, clearly explain which event cannot be scheduled, showing both the 'proposed' time and the overlapping event(s) from 'conflicts_with'.\n"
     "       - Offer to use the `fallback_planner` tool to search for alternative free times for each conflicting event.\n"
     "       - Example: 'Would you like me to suggest free alternative times for the conflicting events, or should I skip them?'\n"
     "       - Only proceed to rescheduling if the user confirms.\n"
     "       - When rescheduling, always use *all busy events for the same date* from `all_events_by_date[proposed['date']]` (not just the conflicts_with list) as the `existing` argument for `fallback_planner`.\n"
     "       - If using `fallback_planner`, show the suggested alternatives and ask the user to confirm which to schedule.\n\n"
     " Do not generate your own event proposals. Always call `filter_conflicting_events` after parsing dates."
     "Tool usage:\n"
     "- Only use structured tool calls — never guess event times, durations, or dates.\n"
     "- When calling `fallback_planner`, never pass the full `proposed` dictionary.\n"
     "- Instead, pass only:\n"
     "    - `context_id`: extracted from `conflicting_blocks[i]['proposed']['context_id']`. This is a UUID.\n"
     "    - `existing`: all busy events for the same date, including both conflicts and accepted_blocks.\n"
     "    - `accepted_blocks`: events already confirmed for the same date. Must not be overlapped.\n"
     "    - `blocked`: optional list of previously suggested fallback slots.\n"
     "- The tool will automatically load the correct proposed event from memory using the `context_id`.\n"
     "- Never regenerate or guess any event fields — always rely on memory.\n"
     "- Always include same-day accepted_blocks in `existing` to prevent overlaps.\n"
     "- Do NOT change the event title or date — keep it exactly as originally proposed.\n"
     "- Do NOT assume or override today's date.\n"
     "- Example call:\n"
     "    fallback_planner({{{{\n"
     "        \"context_id\": \"671be39f-9b87-467f-aa63-65d0274e088d\",\n"
     "        \"existing\": [\n"
     "            {{{{\"start\": \"2025-06-19T08:15:00+02:00\", \"end\": \"2025-06-19T12:15:00+02:00\"}}}},\n"
     "            {{{{\"start\": \"2025-06-19T14:00:00+02:00\", \"end\": \"2025-06-19T19:30:00+02:00\"}}}}\n"
     "        ],\n"
     "        \"accepted_blocks\": [\n"
     "            {{{{\"title\": \"Deep Work\", \"date\": \"19-06-2025\", \"time\": \"12:00\", \"duration_minutes\": 60}}}}\n"
     "        ],\n"
     "        \"blocked\": []\n"
     "    }}}})\n"
     "Important:\n"
     "- Think step-by-step and explain your reasoning if needed.\n"
     "- Only use data returned from tool calls for decisions. Never assume, never skip steps.\n"
    ),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])



#####

In [ ]:
#1. Define tools

tools = [parse_natural_dates, filter_conflicting_events, fallback_planner, create_calendar_events]

#2. Create the agent with custom prompt

agent = create_openai_functions_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

#3. Wrap it into the executor
calendar_agent = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)


#####

In [ ]:
#User interaction cell with Agent
#Make sure the prompt guides the LLM to generate a schedule!

user_prompt = {"input": "Schedule 2 workouts and a coding session for this weekend. Duration is up to you."}

response = calendar_agent.invoke(user_prompt)

#####



> Entering new AgentExecutor chain...

Invoking: `parse_natural_dates` with `{'phrases': ['this weekend']}`


{'parsed_dates': {'this weekend': ['28-06-2025', '29-06-2025']}}
Invoking: `filter_conflicting_events` with `{'events': [{'title': 'Workout', 'date': '28-06-2025', 'time': '10:00', 'duration_minutes': 60}, {'title': 'Workout', 'date': '29-06-2025', 'time': '10:00', 'duration_minutes': 60}, {'title': 'Coding Session', 'date': '28-06-2025', 'time': '14:00', 'duration_minutes': 120}]}`


🛠️ Received input in filter_conflicting_events: [{'title': 'Workout', 'date': '28-06-2025', 'time': '10:00', 'duration_minutes': 60}, {'title': 'Workout', 'date': '29-06-2025', 'time': '10:00', 'duration_minutes': 60}, {'title': 'Coding Session', 'date': '28-06-2025', 'time': '14:00', 'duration_minutes': 120}]
✅ 3 events passed conflict check
[DEBUG] Final conflicting_blocks: []
{'has_conflicts': False, 'accepted_blocks': [{'title': 'Workout', 'date': '28-06-2025', 'time': '10:00', 'duration_min

In [ ]:
# User follow-up prompt (after agent lists conflicts)
#user_prompt = {"input": "Sounds good, please add them to my calendar"}

#Call the agent again with the follow-up user input
#response = calendar_agent.invoke(user_prompt)




> Entering new AgentExecutor chain...

Invoking: `create_calendar_events` with `{'events': [{'title': 'Coding Session 1', 'date': '07-07-2025', 'time': '14:00', 'duration_minutes': 90}, {'title': 'Coding Session 2', 'date': '08-07-2025', 'time': '14:00', 'duration_minutes': 90}, {'title': 'Coding Session 3', 'date': '09-07-2025', 'time': '14:00', 'duration_minutes': 90}, {'title': 'Workout 1', 'date': '07-07-2025', 'time': '08:00', 'duration_minutes': 60}, {'title': 'Workout 2', 'date': '08-07-2025', 'time': '08:00', 'duration_minutes': 60}, {'title': 'Reading Session 1', 'date': '07-07-2025', 'time': '19:00', 'duration_minutes': 45}, {'title': 'Reading Session 2', 'date': '08-07-2025', 'time': '19:00', 'duration_minutes': 45}, {'title': 'Reading Session 3', 'date': '09-07-2025', 'time': '19:00', 'duration_minutes': 45}, {'title': 'Reading Session 4', 'date': '10-07-2025', 'time': '19:00', 'duration_minutes': 45}]}`


['✅ Created: Coding Session 1 — 07-07-2025 at 14:00 for 90 min', '

In [ ]:
# User follow-up prompt (after agent lists conflicts)
#user_prompt = {"input": "Yes, create the event"}

#Call the agent again with the follow-up user input
#response = calendar_agent.invoke(user_prompt)